# This is for the Data Toolkit session

In [2]:
# import np, pd and matplotlib and seaborn
import numpy as np  # type: ignore
import pandas as pd  # type: ignore
import matplotlib.pyplot as plt  # type: ignore
import seaborn as sns  # type: ignore

## Data cleaning

In [3]:
df = pd.read_csv(
    r"C:\ZTalJaZ\Storage\MasterOfDataScience\Data-Sources\Data_Toolkits\Bank_Dataset\bank_marketing_updated_v1.csv",
    header=2, # or skiprows=2 for skipping the first two rows
    low_memory=False,
)

df.head(3)

,customerid,age,salary,balance,marital,jobedu,targeted,default,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,response
0,1,58.0,100000,2143,married,"management,tertiary",yes,no,yes,no,unknown,5,"may, 2017",261 sec,1,-1,0,unknown,no
1,2,44.0,60000,29,single,"technician,secondary",yes,no,yes,no,unknown,5,"may, 2017",151 sec,1,-1,0,unknown,no
2,3,33.0,120000,2,married,"entrepreneur,secondary",yes,no,yes,yes,unknown,5,"may, 2017",76 sec,1,-1,0,unknown,no


### Fixing Rows, Columns

In [24]:
# Drop customerid column
df.drop(columns=["customerid"], inplace=True, axis=1)

df.head(3)

,age,salary,balance,marital,jobedu,targeted,default,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,response
0,58.0,100000,2143,married,"management,tertiary",yes,no,yes,no,unknown,5,"may, 2017",261 sec,1,-1,0,unknown,no
1,44.0,60000,29,single,"technician,secondary",yes,no,yes,no,unknown,5,"may, 2017",151 sec,1,-1,0,unknown,no
2,33.0,120000,2,married,"entrepreneur,secondary",yes,no,yes,yes,unknown,5,"may, 2017",76 sec,1,-1,0,unknown,no


In [ ]:
# Divide the jobedu into 2 different columns: job and education

df[['job', 'education']] = df['jobedu'].str.split(',', expand=True) # expand=True creates new columns
df.drop(columns=["jobedu"], inplace=True, axis=1)

df.head(3)

,age,salary,balance,marital,targeted,default,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,response,job,education
0,58.0,100000,2143,married,yes,no,yes,no,unknown,5,"may, 2017",261 sec,1,-1,0,unknown,no,management,tertiary
1,44.0,60000,29,single,yes,no,yes,no,unknown,5,"may, 2017",151 sec,1,-1,0,unknown,no,technician,secondary
2,33.0,120000,2,married,yes,no,yes,yes,unknown,5,"may, 2017",76 sec,1,-1,0,unknown,no,entrepreneur,secondary


In [36]:
# Check for missing value
df['month'].isnull().sum()

np.int64(50)

In [28]:
# Divide the month and year into 2 different columns: month and year
df['month1'] = df['month'].str.split(',', expand=True)[0]
df['year'] = df['month'].str.split(',', expand=True)[1]
df.drop(columns=['month'], inplace=True)

df.head(5)

,age,salary,balance,marital,targeted,default,housing,loan,contact,day,duration,campaign,pdays,previous,poutcome,response,job,education,month1,year
0,58.0,100000,2143,married,yes,no,yes,no,unknown,5,261 sec,1,-1,0,unknown,no,management,tertiary,may,2017
1,44.0,60000,29,single,yes,no,yes,no,unknown,5,151 sec,1,-1,0,unknown,no,technician,secondary,may,2017
2,33.0,120000,2,married,yes,no,yes,yes,unknown,5,76 sec,1,-1,0,unknown,no,entrepreneur,secondary,may,2017
3,47.0,20000,1506,married,no,no,yes,no,unknown,5,92 sec,1,-1,0,unknown,no,blue-collar,unknown,may,2017
4,33.0,0,1,single,no,no,no,no,unknown,5,198 sec,1,-1,0,unknown,no,unknown,unknown,may,2017


### Impute/Remove Missing Values

In [4]:
# Handling missing values
df['age'].isnull().sum()

np.int64(20)

In [ ]:
float(100*20/df.shape[0]) # 20% of the data is missing

0.04423702196368141

In [ ]:
# Drop records with missing age
ageNotNull = df[~df['age'].isnull()].copy() # this remove all the records with missing age

ageNotNull.shape

(45191, 19)

In [9]:
# Handling missing values in month

# Find numbers of missing data in month
df['month'].isnull().sum()

np.int64(50)

In [11]:
df['month'].value_counts(normalize=True) # check the distribution of month values

month
may, 2017    0.304400
jul, 2017    0.152521
aug, 2017    0.138172
jun, 2017    0.118133
nov, 2017    0.087863
apr, 2017    0.064901
feb, 2017    0.058590
jan, 2017    0.031044
oct, 2017    0.016342
sep, 2017    0.012754
mar, 2017    0.010540
dec, 2017    0.004739
Name: proportion, dtype: float64

In [12]:
# Fill the missing values with the most frequent value
df['month'].fillna(df['month'].mode()[0], inplace=True) # fillna with the most frequent value
df['month'].isnull().sum() # check if there are still missing values in month

np.int64(0)

In [ ]:
df['pdays'].describe()
# there are many -1 values in pdays, which means that there is no contact with the customer, so we can drop these records(missing values)

count    45211.000000
mean        40.197828
std        100.128746
min         -1.000000
25%         -1.000000
50%         -1.000000
75%         -1.000000
max        871.000000
Name: pdays, dtype: float64